In [1]:
!nvidia-smi

Mon Feb 10 06:30:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   61C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os


In [3]:
!pip install ninja

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 9.3 MB/s eta 0:00:00


In [17]:
!pip install .

Processing /content
  Preparing metadata (setup.py) ... done
  Created wheel for custom_kernels: filename=custom_kernels-0.0.1-cp311-cp311-linux_x86_64.whl size=2588545 sha256=ed4e7b2463f26ad5d25b4de03209da2901737b25bb041d09d3b8fea31e30018b
  Stored in directory: /tmp/pip-ephem-wheel-cache-f77tdqlx/wheels/01/d1/e4/ca90c6fac4331f6da6de5353843d0b67505c2bbc8768ac296e
Successfully built custom_kernels
  Attempting uninstall: custom_kernels
    Found existing installation: custom_kernels 0.0.1
    Uninstalling custom_kernels-0.0.1:
      Successfully uninstalled custom_kernels-0.0.1


In [2]:
import torch
import time
import gpu_kernels  # This is your custom PyTorch extension module

import torch
import time

def relu_cpu(input_tensor):
    """CPU-based ReLU implementation using pure Python."""
    output_tensor = torch.empty_like(input_tensor)
    input_np = input_tensor.numpy()
    output_np = output_tensor.numpy()

    for i in range(input_np.shape[0]):
        for j in range(input_np.shape[1]):
            for k in range(input_np.shape[2]):
                output_np[i, j, k] = max(0.0, input_np[i, j, k])

    return torch.from_numpy(output_np)

def benchmark_relu():
    # Create a random input tensor with float32 dtype
    input_tensor = torch.randn(512, 512, 3, dtype=torch.float32).contiguous()
    output_tensor = torch.empty_like(input_tensor)

    # Measure PyTorch ReLU time
    start_time = time.time()
    pytorch_output = torch.nn.functional.relu(input_tensor)
    pytorch_time = time.time() - start_time

    # Measure custom GPU ReLU time
    start_time = time.time()
    gpu_kernels.relu_custom(input_tensor, output_tensor)
    custom_time = time.time() - start_time

    # Measure CPU ReLU time
    start_time = time.time()
    cpu_output = relu_cpu(input_tensor)
    cpu_time = time.time() - start_time

    # Validate correctness
    if not torch.allclose(output_tensor, pytorch_output):
        print("Warning: Custom ReLU output differs from PyTorch's output!")

    if not torch.allclose(cpu_output, pytorch_output):
        print("Warning: CPU ReLU output differs from PyTorch's output!")

    print(f"PyTorch ReLU time: {pytorch_time:.6f} seconds")
    print(f"Custom GPU ReLU time: {custom_time:.6f} seconds")
    print(f"CPU ReLU time: {cpu_time:.6f} seconds")

benchmark_relu()



PyTorch ReLU time: 0.001861 seconds
Custom GPU ReLU time: 0.004157 seconds
CPU ReLU time: 1.654886 seconds
